In [1]:
# Imports 
import os
import glob2 as glob
import cv2
import numpy as np
import matplotlib.pyplot as plt

from skimage.transform import rotate
from skimage.feature import local_binary_pattern
from skimage import data
from skimage.color import label2rgb

In [21]:
dataset_folder = 'dataset\\Splited' # INSERT THE PATH TO THE DATASET

# Define the paths to the train and test folders
train_folder = os.path.join(dataset_folder, 'train')
test_folder = os.path.join(dataset_folder, 'test')

# Retrieve the list of file names in the train folder
train_classes_path = glob.glob(os.path.join(train_folder, "*"))

# Retrieve the list of file names in the test folder
test_classes_path = glob.glob(os.path.join(test_folder, "*"))

In [25]:
train_classes_path[0]

'dataset\\Splited\\train\\KTH_aluminium_foil'

In [ ]:
# Retrieve the list of classes
glob.glob(os.path.join(train_classes_path[0], "*"))

In [ ]:
# settings for LBP
radius = 3
n_points = 8 * radius
METHOD = 'uniform'



In [7]:
train_histograms

### Distance Metrics

In [10]:
# Kullback-Leibler Divergence (KL Divergence)
def kl_divergence(hist_P, hist_Q):
    # Ensure histograms are not zero
    hist_P[hist_P == 0] = 1e-10
    hist_Q[hist_Q == 0] = 1e-10
    
    # Calculate the KL Divergence
    divergence = np.sum(hist_P * np.log(hist_P / hist_Q))
    
    return divergence

In [11]:
# Chi-square distance
def chi_square_distance(hist_P, hist_Q):
    # Ensure histograms are not zero
    hist_P[hist_P == 0] = 1e-10
    hist_Q[hist_Q == 0] = 1e-10
    
    # Calculate the Chi-square distance
    distance = np.sum((hist_P - hist_Q)**2 / (hist_P + hist_Q))
    
    return distance

## Calculate LBP

In [26]:
# custom implementation based on the adjacent negighbours
def calculate_lbp(image):
    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Initialize the LBP image
    lbp_image = np.zeros_like(gray)
    
    # Define 8 neighbors for each pixel
    neighbors = [(0, 1), (-1, 1), (-1, 0), (-1, -1), (0, -1), (1, -1), (1, 0), (1, 1)]
    
    # Iterate over each pixel in the image
    for i in range(1, gray.shape[0] - 1):
        for j in range(1, gray.shape[1] - 1):
            # Get the binary pattern
            binary_pattern = ''
            center_value = gray[i, j]
            for dx, dy in neighbors:
                neighbor_value = gray[i + dy, j + dx]
                binary_pattern += '1' if neighbor_value >= center_value else '0'
            
            # Convert binary pattern to decimal
            lbp_value = int(binary_pattern, 2)
            
            # Assign LBP value to corresponding pixel in LBP image
            lbp_image[i, j] = lbp_value
    
    return lbp_image